## Bibliotecas:

In [1]:
#importanto bibliotecas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, preprocessing

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler
from datetime import datetime

from sklearn.linear_model import Lasso, LassoCV
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split

#### Lendo csv e removendo algumas colunas redundantes:

In [2]:
Y_train=pd.read_csv('shared/bases_budokai_ufpr/produtividade_soja_modelagem.csv')
df=pd.read_csv('shared/bases_budokai_ufpr/agroclimatology_budokai.csv')

In [3]:
Y_train.head()

,nivel,codigo_ibge,name,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,MU,4100103,Abatiá,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,MU,4100459,Altamira do Paraná,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,MU,4100608,Alto Paraná,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,MU,4100707,Alto Piquiri,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,MU,4100509,Altônia,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


In [4]:
Y_train['nivel'].unique()

array(['MU'], dtype=object)

In [5]:
#Y_codigo=Y_train['codigo_ibge']

Y_train=Y_train.drop(columns=['nivel', 'name'])

In [6]:
Y_train.head()

,codigo_ibge,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,4100103,2600.0,1750.0,2100.0,2600.0,2600.0,2080.0,2590.0,3350.0,2320.0,3100.0,1460.0,2640.0,2760.0,3480.0
1,4100459,3100.0,2460.0,2950.0,2950.0,2749.0,2749.0,3250.0,3200.0,2197.0,3223.0,3471.0,3099.0,2871.0,3818.0
2,4100608,1487.0,1365.0,1734.0,2727.0,2400.0,1620.0,2899.0,2899.0,1737.0,2880.0,2000.0,1800.0,2221.0,2980.0
3,4100707,2300.0,2300.0,2700.0,3193.0,2701.0,2097.0,3130.0,3310.0,2000.0,3000.0,3192.0,3488.0,2589.0,3700.0
4,4100509,1050.0,1800.0,1255.0,2355.0,2600.0,1300.0,3000.0,2773.0,815.0,2137.0,1800.0,2455.0,2780.0,2900.0


### De acordo com as respostas esperadas temos:

In [7]:
codigo_saida=[4102000,4104303,4104428,4104808,4104907,4109401,4113205,4113700,4113734,4114005,4114401,4117701,4117909,4119608,4119905,4127007,4127403,4127502,4127700,4128005
]

In [8]:
dataset=pd.DataFrame(columns=df.keys())
#dataset.loc[dataset['codigo_ibge']==codigo_saida[j]]

for k in codigo_saida:
    dataset=pd.concat([df.loc[df['codigo_ibge']==k],dataset])

dataset=dataset.reset_index(drop=True)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131500 entries, 0 to 131499
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   data                 131500 non-null  object 
 1   codigo_ibge          131500 non-null  object 
 2   latitude             131500 non-null  float64
 3   longitude            131500 non-null  float64
 4   T2M_RANGE            131500 non-null  float64
 5   TS                   131500 non-null  float64
 6   T2MDEW               131500 non-null  float64
 7   T2MWET               131500 non-null  float64
 8   T2M_MAX              131500 non-null  float64
 9   T2M_MIN              131500 non-null  float64
 10  T2M                  131500 non-null  float64
 11  QV2M                 131500 non-null  float64
 12  RH2M                 131500 non-null  float64
 13  PRECTOTCORR          131500 non-null  float64
 14  PS                   131500 non-null  float64
 15  WS2M             

In [10]:
dataset['codigo_ibge']=list(map(int,dataset['codigo_ibge']))
#transformar o codigo em inteiro novamente
#df.loc[df['codigo_ibge']==codigo_saida[0]]

### Vamos começar transformando os dados de data em 'datetime' para melhor agrupar em semanas, meses ou anos.

In [11]:
def t_data_day(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.day

def t_data_month(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.month

def t_data_year(data):
    data=str(data)
    data=datetime.strptime(data,"%Y%m%d")
    return data.year
    #return data.day, data.month, data.year

In [12]:
dataset['day']=list(map(t_data_day, dataset['data']))
dataset['month']=list(map(t_data_month, dataset['data']))
dataset['year']=list(map(t_data_year, dataset['data']))

In [13]:
dataset.head()

,data,codigo_ibge,latitude,longitude,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,day,month,year
0,20030101,4128005,-24.5393,-52.9865,7.99,26.27,23.23,24.83,30.21,22.21,...,30.59,42.56,133.74,165.12,18.27,0.56,2.95,1,1,2003
1,20030102,4128005,-24.5393,-52.9865,6.47,25.70,23.36,24.59,29.33,22.86,...,29.79,42.55,111.66,161.13,15.38,0.48,2.55,2,1,2003
2,20030103,4128005,-24.5393,-52.9865,6.27,24.75,22.78,23.78,28.03,21.76,...,29.66,42.53,89.88,161.48,12.85,0.41,2.12,3,1,2003
3,20030104,4128005,-24.5393,-52.9865,6.38,24.50,22.21,23.29,27.82,21.44,...,28.35,42.51,87.03,153.10,12.10,0.40,2.05,4,1,2003
4,20030105,4128005,-24.5393,-52.9865,8.43,25.62,22.50,23.95,29.68,21.24,...,29.04,42.49,52.05,156.25,7.56,0.24,1.24,5,1,2003


## *Uma feature importante é o valor da produção do ano passado, inserir depois.

In [14]:
dataset=dataset.drop(dataset.loc[dataset['year']==2003].index).reset_index(drop=True)

#### A ideia posteriormente aqui é fazer um algoritmo de pesos para cada mês (media ponderada)

### A titulo de teste do sistema podemos implementar um algoritmo rápido para previsões baseado na média dos valores anuais.

In [15]:
dataset=dataset.drop(columns=['data','latitude','longitude','day','month'])

#### Para cada codigo postal agrupamos os dados anuais pela média.

In [16]:
codigos=dataset['codigo_ibge'].unique()

datanew=pd.DataFrame(columns=dataset.keys())

for i in codigos:
    
    aux = dataset.loc[dataset['codigo_ibge']==i].groupby(by='year').agg('mean').reset_index()
    
    datanew=pd.concat([datanew,aux])
    
datanew=datanew.reset_index(drop=True)

In [17]:
datanew.head()

,codigo_ibge,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,QV2M,RH2M,...,GWETPROF,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,year
0,4128005,12.134344,22.684918,16.481120,19.374126,28.814317,16.679617,22.267459,12.478607,73.516858,...,0.648934,18.791585,23.478060,33.443224,99.539836,122.907459,13.052732,0.361257,-55.554290,2004
1,4128005,12.513863,23.570932,16.521342,19.816466,29.874493,17.360466,23.111616,12.565178,70.271068,...,0.636000,18.484247,23.120438,33.413014,98.403945,121.421315,12.859562,0.353014,-31.074164,2005
2,4128005,12.712055,23.623096,16.216822,19.723890,30.100795,17.388575,23.230055,12.531781,68.472986,...,0.624164,18.872740,23.481315,33.410521,100.336000,123.347479,13.155205,0.368658,-61.136137,2006
3,4128005,12.488740,23.495342,16.563041,19.815233,29.797123,17.309315,23.066301,12.713041,70.666192,...,0.683918,19.022082,23.241507,33.408795,100.933973,121.954658,13.145616,0.361863,-47.480521,2007
4,4128005,12.562213,23.200710,16.192596,19.438798,29.450355,16.888169,22.684071,12.274863,70.443361,...,0.631339,18.744399,23.444016,33.432486,100.019945,123.603798,13.145246,0.368716,-50.036530,2008


In [18]:
X_train=datanew.loc[datanew['year']<2018]
X_test=datanew.loc[datanew['year']>2017]


#### Scalando os dados:

In [19]:
colunas_=list(X_train.keys())

colunas_.remove('year')
colunas_.remove('codigo_ibge')
#removendo os dados que não serão escalados

In [20]:
scaler=StandardScaler()


X_train_scaled=scaler.fit_transform(X_train[colunas_])
X_train_scaled=pd.DataFrame(X_train_scaled, columns=colunas_)


X_test_scaled=scaler.transform(X_test[colunas_])
X_test_scaled=pd.DataFrame(X_test_scaled, columns=colunas_)

### Precisamos reinserir o codigo e ano e resetar o indice.

In [21]:
X_train_scaled[['codigo_ibge','year']]=X_train[['codigo_ibge','year']].reset_index(drop=True)
X_test_scaled[['codigo_ibge','year']]=X_test[['codigo_ibge','year']].reset_index(drop=True)

In [22]:
X_train_scaled

,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,QV2M,RH2M,PRECTOTCORR,...,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,TOA_SW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,codigo_ibge,year
0,1.150433,1.068194,0.589315,0.863577,1.089748,0.808277,0.986354,0.204803,-0.998920,-0.114293,...,1.191969,1.229751,1.340536,1.132241,0.899263,1.172491,1.241870,-1.450284,4128005,2004
1,1.538126,1.481150,0.620736,1.145916,1.549881,1.205817,1.412928,0.320407,-1.736024,-1.037937,...,0.860170,-0.349586,1.258387,0.910232,-0.051191,0.854623,0.795989,0.033954,4128005,2005
2,1.740586,1.505463,0.382849,1.086826,1.648099,1.222230,1.472778,0.275810,-2.144360,-0.710439,...,1.279583,1.244126,1.251607,1.287850,1.180675,1.341114,1.642184,-1.788713,4128005,2006
3,1.512461,1.445919,0.653311,1.145129,1.516301,1.175951,1.390029,0.517859,-1.646293,-1.231837,...,1.440811,0.185080,1.246914,1.404723,0.289905,1.325335,1.274660,-0.960769,4128005,2007
4,1.587517,1.308596,0.363924,0.904856,1.365799,0.930049,1.196878,-0.067269,-1.696897,-1.309586,...,1.141028,1.079407,1.311338,1.226077,1.344602,1.324725,1.645338,-1.115740,4128005,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,0.269349,-0.028614,0.352136,0.145611,0.046403,-0.091663,0.002739,0.293078,0.354020,-0.016966,...,0.372995,1.009885,0.058791,0.372840,1.056411,0.412179,0.566286,-1.463010,4102000,2013
276,0.502036,0.398995,0.400720,0.448368,0.514558,0.405330,0.450429,0.338115,-0.328293,-0.748219,...,0.002356,-0.248920,0.055066,0.081595,0.263166,0.065672,-0.183581,0.526449,4102000,2014
277,-0.550035,0.441083,1.386899,0.903980,0.361784,0.800950,0.534064,1.671651,0.659249,1.422411,...,-1.187941,-2.085343,0.076298,-0.996302,-1.061320,-1.186868,-0.917148,1.354203,4102000,2015
278,-0.308298,-0.149148,0.669889,0.216591,-0.163478,-0.043865,-0.090347,0.858548,0.928557,0.342120,...,0.110729,-0.029355,0.121706,0.097901,0.095447,0.054881,0.325570,-1.124487,4102000,2016


In [23]:
#k-fold para achar melhor valor de alpha
model = LassoCV(cv=5, random_state=0, max_iter=10000)

In [24]:


df_out=pd.DataFrame(columns=['codigo_ibge','2018','2019','2020'])

for i in codigo_saida:
    
    X_train_menor=X_train_scaled.loc[X_train_scaled['codigo_ibge']==i].drop(columns=['codigo_ibge','year'])
    Y_train_menor=Y_train[Y_train['codigo_ibge']==i].drop(columns='codigo_ibge').T
    
    model.fit(X_train_menor, Y_train_menor)
    ##########################
    # Teste com árvore de decisão
    dt = DecisionTreeRegressor(max_depth=4, min_samples_leaf=0.1, random_state=3)
    dt.fit(X_train_menor,Y_train_menor)
    ##############################################3
    lasso_t= Lasso(alpha=model.alpha_, max_iter= 10000).fit(X_train_menor,Y_train_menor)
    
    print(f'\n Alpha usado: {model.alpha_}')
    print('Features non-zero para {}: {}'.format(i, np.sum(lasso_t.coef_ != 0)))
    print('Features com valores non-zero (ordenados pela magnitude absoluta):')

    for e in sorted (list(zip(list(X_train), lasso_t.coef_)), key = lambda e: -abs(e[1])):
        if e[1] != 0:
            print('\t{}, {:.3f}'.format(e[0], e[1]))
    
    
    X_test_menor=X_test_scaled.loc[X_test_scaled['codigo_ibge']==i].drop(columns=['codigo_ibge','year'])
    ##################
    Y_prd = dt.predict(X_test_menor)
    predicoes = Y_prd
    #Analisando a acuracia
    ##################
    #predicoes=lasso_t.predict(X_test_menor)

    data_saida={'codigo_ibge':i, '2018':predicoes[0], '2019':predicoes[1], '2020':predicoes[2]}
    
    data_saida=pd.DataFrame([data_saida])
    
    df_out=pd.concat([df_out,data_saida])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 42.67871104220746
Features non-zero para 4102000: 4
Features com valores non-zero (ordenados pela magnitude absoluta):
	QV2M, 1249.771
	RH2M, -707.274
	ALLSKY_SFC_UVA, -217.895
	ALLSKY_SFC_UVB, 155.427

 Alpha usado: 80.35273770572255
Features non-zero para 4104303: 2
Features com valores non-zero (ordenados pela magnitude absoluta):
	RH2M, 121.380
	QV2M, 58.410

 Alpha usado: 34.60790701737417
Features non-zero para 4104428: 5
Features com valores non-zero (ordenados pela magnitude absoluta):
	WS50M_MAX, 426.089
	WS50M_RANGE, 263.062
	RH2M, -126.649
	T2M, 117.311
	ALLSKY_SFC_PAR_TOT, 89.845


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 35.40548124150876
Features non-zero para 4104808: 5
Features com valores non-zero (ordenados pela magnitude absoluta):
	QV2M, 262.773
	T2M, 170.308
	ALLSKY_SFC_PAR_TOT, 169.564
	WS50M_MAX, 43.240
	ALLSKY_SFC_UVB, 35.663


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 9.941968770462411
Features non-zero para 4127007: 6
Features com valores non-zero (ordenados pela magnitude absoluta):
	TOA_SW_DWN, 473.616
	GWETPROF, 272.221
	RH2M, 246.002
	T2M, 230.350
	ALLSKY_SFC_UVB, -55.951
	QV2M, 44.742

 Alpha usado: 260.30083811652503
Features non-zero para 4127403: 0
Features com valores non-zero (ordenados pela magnitude absoluta):


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Alpha usado: 222.74646386888594
Features non-zero para 4127502: 1
Features com valores non-zero (ordenados pela magnitude absoluta):
	RH2M, 0.000

 Alpha usado: 64.92472189637034
Features non-zero para 4127700: 2
Features com valores non-zero (ordenados pela magnitude absoluta):
	QV2M, 427.771
	ALLSKY_SFC_UVB, -189.952

 Alpha usado: 67.30236334804032
Features non-zero para 4128005: 1
Features com valores non-zero (ordenados pela magnitude absoluta):
	QV2M, 230.123


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
df_out=df_out.reset_index(drop=True)

In [26]:
df_out

,codigo_ibge,2018,2019,2020
0,4102000,1530.500000,3302.250000,2066.666667
1,4104303,3043.000000,3707.000000,2798.000000
2,4104428,2328.500000,2898.666667,2898.666667
3,4104808,3583.666667,2938.000000,2554.000000
4,4104907,2789.000000,2789.000000,3421.000000
5,4109401,3006.333333,3006.333333,3006.333333
6,4113205,3055.000000,2357.500000,2357.500000
7,4113700,2190.000000,2521.500000,2190.000000
8,4113734,2480.000000,3431.500000,3065.500000
9,4114005,2621.000000,3403.500000,2621.000000


In [27]:
df_out.to_csv('submission.csv',index=False)

PermissionError: [Errno 13] Permission denied: 'submission.csv'